In [5]:
import configparser
import os
import requests
import pandas as pd
from IPython.display import display, clear_output, Video, HTML
from urllib.parse import urlencode

In [6]:
class ADSSearch:
    
    def __init__(self):
        self.api_token = os.environ.get("ADS_DEV_KEY")

        if not self.api_token:
            config = configparser.ConfigParser()
            config.read('config.ini')
            self.api_token = config.get('DEFAULT', 'ADS_DEV_KEY', fallback=None)

        if not self.api_token:
            raise ValueError("ADS API key not found. Please set the ADS_DEV_KEY environmental variable or add it to the config.ini file.")
    
    def search(self, search_terms):
        headers = {"Authorization": f"Bearer {self.api_token}"}

        # Construct the query string
        query_terms = " AND ".join(f"full:\"{term}\"" for term in search_terms)
        query = f"{query_terms}"
        
        # Set up the query parameters
        params = {
            "q": query,
            "fl": "id,title,bibcode,author,year",
            "rows": 100,
            "sort": "date desc"
        }

        # Make the API request
        response = requests.get("https://api.adsabs.harvard.edu/v1/search/query", headers=headers, params=params)
        response_json = response.json()
        #print(response_json)
        # Process the response and return the results
        results = []
        for paper in response_json["response"]["docs"]:
            result = {
                "title": paper["title"][0],
                "bibcode": paper["bibcode"],
                "first_author": paper["author"][0],
                "year": paper["year"],
                "url": f"https://ui.adsabs.harvard.edu/abs/{paper['bibcode']}"
            }
            results.append(result)
        return results
    
    def search_advanced(self, query_params):
        headers = {"Authorization": f"Bearer {self.api_token}"}

        # Encode the query parameters
        encoded_query = urlencode(query_params)

        # Make the API request
        response = requests.get(f"https://api.adsabs.harvard.edu/v1/search/query?{encoded_query}", headers=headers)
        response_json = response.json()

        # Check for the 'response' key in the response JSON
        if "response" not in response_json:
            raise ValueError("Unexpected API response. 'response' key not found in the response JSON.")

        # Process the response and return the results
        results = []
        for paper in response_json["response"]["docs"]:
            result = {k: v for k, v in paper.items() if v}  # Filter out empty fields
            result["url"] = f"https://ui.adsabs.harvard.edu/abs/{paper['bibcode']}"
            results.append(result)
        return results


In [3]:
ads = ADSSearch()

# Search for papers with the given search terms
search_terms = ["SST", "CRISP", "25 May 2017"]
results = ads.search(search_terms)

# Print the paper titles, bibcodes, and URLs
# for result in results:
#     print(result["title"])
#     print(result["bibcode"])
#     print(result["first_author"])
#     print(result["year"])
#     print(result["url"])
headers = ["#", "Title", "First Author", "Bibcode", "URL"]
rows = [[i+1, result["title"],result["first_author"], result["bibcode"], result["url"]] for i, result in enumerate(results)]
df = pd.DataFrame(rows, columns=headers)
if len(rows) > 0:
    # Generate the HTML table with links and disable HTML escaping
    html_table = df.to_html(render_links=True, escape=False, index=False)

    # Define CSS style rules for the table cells and header cells
    cell_style = "td { text-align: left; }"
    header_style = "th { text-align: center; }"

    # Use the HTML module to display the table with the style rules
    display(HTML(f'<style>{cell_style} {header_style}</style>{html_table}'))


#,Title,First Author,Bibcode,URL
1,On the dynamics of spicules and mass flows in the solar atmosphere,"Bose, Souvik",2021arXiv211010656B,https://ui.adsabs.harvard.edu/abs/2021arXiv211010656B
2,Evidence of the multi-thermal nature of spicular downflows. Impact on solar atmospheric heating,"Bose, Souvik",2021A&A...654A..51B,https://ui.adsabs.harvard.edu/abs/2021A&A...654A..51B
3,Analysis of Pseudo-Lyapunov Exponents of Solar Convection Using State-of-the-Art Observations,"Viavattene, Giorgio",2021Entrp..23..413V,https://ui.adsabs.harvard.edu/abs/2021Entrp..23..413V
4,Spicules and downflows in the solar chromosphere,"Bose, Souvik",2021A&A...647A.147B,https://ui.adsabs.harvard.edu/abs/2021A&A...647A.147B
5,Characterization and formation of on-disk spicules in the Ca II K and Mg II k spectral lines,"Bose, Souvik",2019A&A...631L...5B,https://ui.adsabs.harvard.edu/abs/2019A&A...631L...5B


In [20]:
author_list = {'author': ['Bose, Souvik', 'Rouppe van der Voort, Luc', 'Joshi, Jayant', 'Henriques, Vasco M. J.', 'Nóbrega-Siverio, Daniel', 'Martínez-Sykora, Juan', 'De Pontieu, Bart']}
author_list['author'][0]
#split the author list into first and last name
first_name = author_list['author'][0].split(',')[1]
last_name = author_list['author'][0].split(',')[0]
print(first_name)
print(last_name)

 Souvik
Bose


In [15]:
import pandas as pd
import pipmag as pm
latest_updated_la_palma_obs_data_file = pm.get_latest_file('data/la_palma_obs_data_*.pkl')
df = pd.read_pickle(latest_updated_la_palma_obs_data_file)

Latest file: data/la_palma_obs_data_20230216_182941.pkl


In [16]:
dt = df.iloc[42].date_time

In [17]:
# convert dt into string in the format 25 May 2017
dt_str = dt.strftime('%d %B %Y')
print(dt_str)

06 June 2019


In [18]:
# convert dt into string in the format 6 June 2019
dt_str = dt.strftime('%-d %B %Y')
print(dt_str)

6 June 2019


In [19]:
#convert the dt into string in the format 25 May 2017 if the date is two digits, otherwise 6 June 2019
dt_str = dt.strftime('%-d %B %Y') if dt.day < 10 else dt.strftime('%d %B %Y')

In [20]:
ins = df.iloc[36].instruments

In [21]:
# combine the list of instruments and dt_str into a list
search_terms = ['SST'] + ins + [dt_str]
print(search_terms)

['SST', 'CRISP', '6 June 2019']


In [22]:
results = ads.search(search_terms)

# Print the paper titles, bibcodes, and URLs
for result in results:
    print(result["title"])
    print(result["bibcode"])
    print(result["url"])

Properties of shock waves in the quiet-Sun chromosphere
2022A&A...668A.153M
https://ui.adsabs.harvard.edu/abs/2022A&A...668A.153M
Properties of ubiquitous magnetic reconnection events in the lower solar atmosphere
2022A&A...664A..72J
https://ui.adsabs.harvard.edu/abs/2022A&A...664A..72J
Signatures of ubiquitous magnetic reconnection in the lower solar atmosphere
2020A&A...641L...5J
https://ui.adsabs.harvard.edu/abs/2020A&A...641L...5J
